,Marks,Time,Score
0,50,1.0,1
1,80,2.0,1
2,82,3.0,1
3,95,1.5,1
4,800,60.0,-1


### Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [ ]:
data =pd.read_csv("iris.csv",index_col=0)

In [ ]:
data.head()

In [ ]:
#Changes does not effect the "data" dataframe
data1=data.copy()

In [ ]:
labelencoder = LabelEncoder() # we want to label the Species column i.e.y variable
data1.iloc[:, -1] = labelencoder.fit_transform(data1.iloc[:,-1]) # -1: consider last column

In [ ]:
data1

### One Hot Encoder

#### Using sklearn

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data2=pd.read_csv("iris.csv",index_col=0)

In [ ]:
data2

In [ ]:
# creating instance of one-hot-encoder
OHE = OneHotEncoder()# Specifies the way unknown categories are handled during transform.

In [ ]:
# convert it to array - to columns
enc_df = pd.DataFrame(OHE.fit_transform(data2[['Species']]).toarray())

In [ ]:
enc_df

In [ ]:
# merge with main df 
data_final = data2.iloc[:,0:4].join(enc_df)
data_final

#### Using Pandas

In [ ]:
import pandas as pd

In [ ]:
data3 =pd.read_csv("iris.csv",index_col=0)

In [ ]:
data_encoded=pd.get_dummies(data3)

In [ ]:
data_encoded

### IsolationForest

In [ ]:
# Isolation Forest example
from sklearn.ensemble import IsolationForest
import pandas as pd

In [ ]:

df=pd.DataFrame({
      'Marks':[50,80,82,95,800],
      'Time':[1,2,3,1.5,60]
})
df

In [ ]:
# what is outlier here?
iso=IsolationForest(random_state=10,contamination=0.1)# expecting 1 % outlier in dataset
iso.fit(df)
out=iso.predict(df)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [ ]:
out # -1: outlier - last value is an outlier

array([ 1,  1,  1,  1, -1])

In [ ]:
df['Score']=out
df

,Marks,Time,Score
0,50,1.0,1
1,80,2.0,1
2,82,3.0,1
3,95,1.5,1
4,800,60.0,-1


In [ ]:
from sklearn.ensemble import IsolationForest
import pandas as pd

In [ ]:
data =pd.read_csv("iris.csv",index_col=0)
data_encoded=pd.get_dummies(data)

In [ ]:
data_encoded

In [ ]:
# u can detect outliers using percentiles as well. Other than this, boxplot you can use for outlier detection.
data_encoded['Petal.Width'].describe(percentiles = [0.01, 0.02,0.9,0.99,0.991,0.992])

count    150.000000
mean       1.199333
std        0.762238
min        0.100000
1%         0.100000
2%         0.100000
50%        1.300000
90%        2.200000
99%        2.500000
99.1%      2.500000
99.2%      2.500000
max        2.500000
Name: Petal.Width, dtype: float64

In [ ]:
# training the model
clf = IsolationForest(random_state=10,contamination=.01)
# contamination: how much percentage of outliers you are expecting in dataset
# eg. in health care domain it will be very low .01 or .001
clf.fit(data_encoded)

In [ ]:
# predictions
y_pred_outliers = clf.predict(data_encoded)

In [ ]:
#-1 for outliers and 1 for inliers.
y_pred_outliers

In [ ]:
data_encoded

In [ ]:
## Let us add a  new data point which is outlier
data_encoded.loc[151]=[20,40,30,50,1,0,0]

In [ ]:
data_encoded

In [ ]:
# training the model
clf = IsolationForest(random_state=10,contamination=.01)
clf.fit(data_encoded)
# predictions
y_pred_outliers = clf.predict(data_encoded)
y_pred_outliers # last has -1 so its an outlier

In [ ]:
# data_encoded['scores']=clf.decision_function(data_encoded)

In [ ]:
data_encoded['anomaly']=clf.predict(data_encoded.iloc[:,0:7])
# we can pinpoint those outliers exactly by applying this filtering score

In [ ]:
data_encoded

In [ ]:
#Print the outlier data points
data_encoded[data_encoded['anomaly']==-1]# scores are given by decision tree

### PPS score

In [ ]:
#install the package
!pip install ppscore

In [ ]:
import ppscore as pps

In [ ]:
# weighted F1 Score concept
#  https://towardsdatascience.com/multi-class-metrics-made-simple-part-ii-the-f1-score-ebe8b2c2ca1

In [ ]:
data.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa


In [ ]:
#pps.score(df, "feature_column", "target_column")  syntax
pps.score(data, "Sepal.Length", "Petal.Length")#ppscore:0.55 so ok kind of score

In [ ]:
#calculate the whole PPS matrix
pps.matrix(data)# petal width with species is most imp varaible with pps score 0.9276

In [ ]:
df=pd.DataFrame(pps.matrix(data))
df[df.case!='predict_itself'].iloc[:,[0,1,2,3,5,8]].sort_values('ppscore',ascending=False)

,x,y,ppscore,case,metric,model
19,Petal.Width,Species,0.927652,classification,weighted F1,DecisionTreeClassifier()
14,Petal.Length,Species,0.884812,classification,weighted F1,DecisionTreeClassifier()
17,Petal.Width,Petal.Length,0.798274,regression,mean absolute error,DecisionTreeRegressor()
22,Species,Petal.Length,0.785393,regression,mean absolute error,DecisionTreeRegressor()
23,Species,Petal.Width,0.755749,regression,mean absolute error,DecisionTreeRegressor()
13,Petal.Length,Petal.Width,0.744945,regression,mean absolute error,DecisionTreeRegressor()
2,Sepal.Length,Petal.Length,0.550423,regression,mean absolute error,DecisionTreeRegressor()
10,Petal.Length,Sepal.Length,0.525617,regression,mean absolute error,DecisionTreeRegressor()
4,Sepal.Length,Species,0.471649,classification,weighted F1,DecisionTreeClassifier()
3,Sepal.Length,Petal.Width,0.431739,regression,mean absolute error,DecisionTreeRegressor()
